In [1]:
import collector

In [6]:
main = collector.Collector('dart')
main._setMaria()
main._setDart()
data = {}
years = ['2017', '2018', '2019', '2020']
for y in years:
    data['fnguide{}'.format(y)] = main.maria.showData('select * from fn{}'.format(y))
code_list = list(data['fnguide2017']['종목코드'])

In [7]:
def checkFS(code, df):
    res = df[df['종목코드'] == code]['FS'].iloc[0]
    return res

In [8]:
def saveData(tablename, data):
    if data is None:
        return
    main.maria.insertData(tablename, data)
    main.maria.commitDB()

In [9]:
def getFloatData(data):
    return [float(d.replace(',',''))/100000000 for d in data]

# Dart - 자본

In [6]:
# def getJBData(code, df):
#     fs = main.getDartFs(code, True, 2019, '11011')
#     if fs is None:
#         print('{}는 api에 데이터가 없습니다'.format(code))
#         main.writeError('error_dart', code, 'api에 데이터가 없음')
#         return fs
#     check_fs = checkFS(code, df)
#     if check_fs == '연결':
#         df = fs[fs['fs_div'] == 'CFS']
#     else:
#         df = fs[fs['fs_div'] == 'OFS']
#     try:
#         jb_data = getFloatData(list(df[df['account_nm'] == '자본금'][['bfefrmtrm_amount', 'frmtrm_amount', 'thstrm_amount']].iloc[0]))
#     except:
#         try:
#             if check_fs == '연결':
#                 df = fs[fs['fs_div'] == 'OFS']
#                 print('{}를 개별에서 가져왔습니다'.format(code))
#                 main.writeError('error_dart', code, '연결 데이터에 없어서 개별 데이터에서 가져옴')
#                 jb_data = getFloatData(list(df[df['account_nm'] == '자본금'][['bfefrmtrm_amount', 'frmtrm_amount', 'thstrm_amount']].iloc[0]))
#             else:
#                 df = fs[fs['fs_div'] == 'CFS']
#                 print('{}를 연결에서 가져왔습니다'.format(code))
#                 main.writeError('error_dart', code, '개별 데이터에 없어서 연결 데이터에서 가져옴')
#                 jb_data = getFloatData(list(df[df['account_nm'] == '자본금'][['bfefrmtrm_amount', 'frmtrm_amount', 'thstrm_amount']].iloc[0]))
#         except:
#             print('{}의 finstate에는 [자본금]이 없습니다'.format(code))
#             main.writeError('error_dart', code, '[자본금]데이터가 없음')
#             return None
#     return data

In [7]:
# done_list = main.getDoneList('new_dart_jb')
# std_df = data['fnguide2017']
# for code in code_list:
#     if code in done_list:
#         continue
#     print(code)
#     jb_data = getJBData(code, std_df)
#     if jb_data is None:
#         continue
#     saveData(tablename='new_dart_jb', data=tuple([code]+jb_data))

# Dart - 무형자산

In [87]:
def getMHData(code, df, crp_list):
    check_fs = checkFS(code, df)
    if check_fs == '연결':
        fs = main.getDartFs(code, False, 2019, '11011')
    else:
        try:
            crp_no = crp_list.find_by_stock_code(code)
            fs_se = crp_no.extract_fs(bgn_de='20170930', end_de='20201231', separate=True)
            cols = fs_se[0].columns
            selected_cols = [c[0] if len(c[0])==8 else c[1] for c in cols]
            fs = fs_se[0]
            fs.columns = selected_cols
        except:
            return
    if fs is None:
        return fs
    
    c_params = [['account_id', 'Intangible assets'],
               ['account_id', 'dart_OtherIntangibleAssetsGross'],
               ['account_id', 'ifrs-full_IntangibleAssetsOtherThanGoodwill'],
               ['account_nm', '무형자산']]
    o_params = [['label_en', 'Intangible assets'],
               ['label_en', 'dart_OtherIntangibleAssetsGross'],
               ['concept_id', 'dart_OtherIntangibleAssetsGross'],
               ['concept_id', 'ifrs-full_IntangibleAssetsOtherThanGoodwill'],
                ['label_ko', '무형자산']
               ]
    if check_fs == '연결':
        mh_data = main.loopDart(code, fs, c_params, 'new_dart_mh', ['bfefrmtrm_amount', 'frmtrm_amount', 'thstrm_amount'])
    else:
        mh_data = main.loopDart(code, fs, o_params, 'new_dart_mh', ['20171231', '20181231', '20191231'])
    if mh_data is None:
        return
    return data

In [88]:
import dart_fss
dart_fss.set_api_key('abeae84cda5efbd4511011450af0c4aa6603d140')
crp_list = dart_fss.get_corp_list()

Output()

Output()

Output()

In [99]:
done_list = main.getDoneList('new_dart_mh')
std_df = data['fnguide2017']
for code in code_list[10:]:
    if code in done_list:
        continue
    print(code)
    mh_data = getMHData(code, std_df, crp_list)
    if mh_data is None:
        continue

000250
001040
001140
001390
001740
001810
002070
002270
002300
002320
002390
002420


Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

002450
002460
002870
002990
003080
003720
003800
003960
004140
004200
004370
004560
004650
004700


Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

004710
005070


...

258540


Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

260970


Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

900040
900070
900110
900140


# 주식수

In [52]:
main.maria.createTable('stocks_20210304', ['종목코드', '보통주', '우선주'], ['VARCHAR(20)', 'FLOAT', 'FLOAT'])

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import collector
import numpy as np

main = collector.Collector('fnguide')
main._setMaria()

webdriver_options = webdriver.ChromeOptions()
webdriver_options.add_argument('headless')
chromedriver = 'chromedriver.exe'
#driver = webdriver.Chrome(chromedriver)
driver = webdriver.Chrome(chromedriver, options=webdriver_options)


main.maria.createTable('stocks_202103', ['종목코드', '보통주', '우선주'], ['VARCHAR(20)', 'FLOAT', 'FLOAT'])
done_list = main.getDoneList('stocks_202103')

for code in main.code_list:
    print(code)
    if code in done_list:
       continue
    try:
        data = {}
        url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701".format(code)
        driver.get(url)
        html = driver.page_source
        df1 = pd.read_html(html)
        data['종목코드'] = code
        data['보통주'] = float(df1[0][1][6].split('/')[0].replace(' ','').replace(',',''))
        data['우선주'] = float(df1[0][1][6].split('/')[1].replace(' ','').replace(',',''))
        main.maria.insertData('stocks_202103', tuple(list(data.values())))
        main.maria.commitDB()
    except:
        pass

005930
000660
005380
005935
015760
012330
035420
028260
032830
005490
055550
090430
017670
105560
051910
034730
000270
033780
096770
051900
000810
011170
068270
034220
009540
002790
018260
003550
207940
010950
086790
010130
000030
066570
030200
035250
004020
006400
161390
024110
023530
021240
047810
086280
088350
018880
001040
036570
035720
004800
139480
078930
032640
000720
097950
009240
029780
036460
005830
000120
009830
027410
001800
002380
009150
005387
007070
010140
241560
008930
012630
012750
128940
047050
000210
034020
138930
001450
026960
130960
005940
088980
000880
069960
004990
011780
051600
016360
005385
006800
037620
071050
012450
007310
000150
000100
204320
090435
028670
047040
011070
004370
028050
086900
003490
011210
000670
000240
016170
041960
006260
036490
008770
006360
010060
120110
006280
042670
030000
005300
006650
079550
004170
008560
001740
000060
139130
084990
006120
039490
001440
138040
002550
003520
064350
005440
079160
000080
005610
014820
017800
161890
003690

161570
019685
007590
241790
009070
036200
131970
080520
004080
133750
094360
088910
084670
156100
049080
059100
036010
084180
147760
080220
031860
004450
066900
058450
015230
038680
049630
069330
011700
042110
079650
160980
053690
047820
050540
024910
099440
216280
030960
049830
003610
092460
001070
039340
045660
058730
108860
021050
085810
229500
092870
013870
123700
016590
002210
092600
066410
100090
023900
095190
123840
207760
019550
000590
069410
045510
054540
083420
048770
006570
056080
038460
144620
036180
018000
078860
000500
001750
006660
025880
033100
083650
187220
120030
032850
012800
038070
095910
052770
019540
208370
036670
105550
117930
047560
178780
043100
014130
226350
001340
008260
092220
066590
068930
036690
008870
084650
007330
011785
052290
142210
131290
094840
037330
063760
006405
008700
110570
079660
079970
039230
065950
028080
023450
052220
032800
024120
010960
120115
068150
155960
214680
217950
053700
012280
018290
006340
033540
017510
019570
032790
059090
133820

In [ ]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import collector
import numpy as np

main = collector.Collector('fnguide')
main._setMaria()

webdriver_options = webdriver.ChromeOptions()
webdriver_options.add_argument('headless')
chromedriver = 'chromedriver.exe'
#driver = webdriver.Chrome(chromedriver)
driver = webdriver.Chrome(chromedriver, options=webdriver_options)


main.maria.createTable('roe_202103', ['종목코드', 'ROE', '우선주'], ['VARCHAR(20)', 'FLOAT', 'FLOAT'])
done_list = main.getDoneList('roe_202103')

for code in main.code_list:
    print(code)
    if code in done_list:
       continue
    try:
        data = {}
        url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701".format(code)
        driver.get(url)
        html = driver.page_source
        df1 = pd.read_html(html)
        data['종목코드'] = code
        data['보통주'] = float(df1[0][1][6].split('/')[0].replace(' ','').replace(',',''))
        data['우선주'] = float(df1[0][1][6].split('/')[1].replace(' ','').replace(',',''))
        main.maria.insertData('stocks_202103', tuple(list(data.values())))
        main.maria.commitDB()
    except:
        pass

# ROE

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import collector
import numpy as np

main = collector.Collector('fnguide')
main._setMaria()

webdriver_options = webdriver.ChromeOptions()
webdriver_options.add_argument('headless')
chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(chromedriver, options=webdriver_options)

main.maria.createTable('roe', ['종목코드', '날짜1', '날짜2', '날짜3', '날짜4', '날짜5',
                              'roe1', 'roe2', 'roe3', 'roe4', 'roe5'], 
                       ['VARCHAR(20)', 'VARCHAR(20)', 'VARCHAR(20)', 'VARCHAR(20)', 'VARCHAR(20)', 'VARCHAR(20)', 
                        'FLOAT', 'FLOAT', 'FLOAT', 'FLOAT','FLOAT'])
done_list = main.getDoneList('roe')
for code in main.code_list:
    if code in done_list:
        continue
    try:
        url = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'.format(code)
        driver.get(url)
        html = driver.page_source
        df_list = pd.read_html(html)
        df = df_list[0]
        dates = list(df.columns[1:])
        dates_to_insert = [d.replace('/', '') for d in dates]
        data = tuple([code] + dates_to_insert + list(df[df[df.columns[0]].str.contains('ROE')][dates].fillna(-9999999).astype('float').iloc[0]))

        main.maria.insertData('roe', data)
        main.maria.commitDB()
    except:
        pass

In [10]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import collector
import numpy as np

main = collector.Collector('fnguide')
main._setMaria()

code = '105560'
webdriver_options = webdriver.ChromeOptions()
webdriver_options.add_argument('headless')
chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(chromedriver, options=webdriver_options)



In [11]:
code = '000020'

In [13]:
df

,IFRS(연결),2016/12,2017/12,2018/12,2019/12,2020/09
0,안정성비율,안정성비율,안정성비율,안정성비율,안정성비율,안정성비율
1,유동비율계산에 참여한 계정 펼치기,273.2,386.2,374.8,422.6,358.6
2,당좌비율계산에 참여한 계정 펼치기,225.3,325.9,302.9,362.6,301.8
3,부채비율계산에 참여한 계정 펼치기,28.3,23.7,24.6,25.0,27.8
4,유보율계산에 참여한 계정 펼치기,812.5,972.2,972.7,986.2,1074.7
5,순차입금비율계산에 참여한 계정 펼치기,NaN,NaN,NaN,NaN,NaN
6,이자보상배율계산에 참여한 계정 펼치기,1461.3,NaN,NaN,47.9,95.3
7,자기자본비율계산에 참여한 계정 펼치기,77.9,80.9,80.3,80.0,78.3
8,성장성비율,성장성비율,성장성비율,성장성비율,성장성비율,성장성비율
9,매출액증가율계산에 참여한 계정 펼치기,6.4,9.0,18.4,0.2,-9.1
